In [2]:
import pandas as pd 

df = pd.read_parquet('../data/movie_data.parquet')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99806 entries, 0 to 99805
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   userId             99806 non-null  int64  
 1   rating             99806 non-null  float64
 2   imdb_id            99806 non-null  object 
 3   adult              99806 non-null  object 
 4   genres             99806 non-null  object 
 5   original_language  99806 non-null  object 
 6   overview           99792 non-null  object 
 7   popularity         99806 non-null  float64
 8   poster_path        99800 non-null  object 
 9   release_date       99800 non-null  object 
 10  runtime            99806 non-null  float64
 11  title              99806 non-null  object 
 12  vote_average       99806 non-null  float64
 13  vote_count         99806 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 10.7+ MB


In [4]:
df.head()

,userId,rating,imdb_id,adult,genres,original_language,overview,popularity,poster_path,release_date,runtime,title,vote_average,vote_count
0,7,3.0,tt0114709,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,5415.0
1,9,4.0,tt0114709,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,5415.0
2,13,5.0,tt0114709,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,5415.0
3,15,2.0,tt0114709,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,5415.0
4,19,3.0,tt0114709,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,5415.0


In [17]:
ratings = df[['userId','rating','title']]
ratings.head()

,userId,rating,title
0,7,3.0,Toy Story
1,9,4.0,Toy Story
2,13,5.0,Toy Story
3,15,2.0,Toy Story
4,19,3.0,Toy Story


In [33]:
ratings.rating.value_counts()

rating
4.0    28702
3.0    20039
5.0    15051
3.5    10509
4.5     7701
2.0     7259
2.5     4443
1.0     3321
1.5     1685
0.5     1096
Name: count, dtype: int64

In [20]:
from surprise import Dataset, Reader

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'title', 'rating']], reader)


In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [22]:
from surprise import SVD
from surprise import accuracy

model = SVD()
model.fit(trainset)

# Evaluate performance on test set
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)


RMSE: 0.8954


In [23]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    # Map predictions to each user
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    
    # Sort predictions by estimated rating
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    
    return top_n


In [31]:
userId =10

In [32]:
trainset = model.trainset
all_items = trainset.all_items()
all_item_inner_ids = [trainset.to_raw_iid(iid) for iid in all_items]
all_users = trainset.all_users()

# Get unseen items per user
def get_unseen_items(trainset, user):
    seen_items = set([j for (j, _) in trainset.ur[user]])
    all_items = set(trainset.all_items())
    return list(all_items - seen_items)

# Generate top-N for userId = 3
user_inner_id = trainset.to_inner_uid(userId)
unseen = get_unseen_items(trainset, user_inner_id)

predictions = [model.predict(uid=userId, iid=trainset.to_raw_iid(iid)) for iid in unseen]

top_n = get_top_n(predictions, n=5)

recommendations = top_n[userId]

titles, scores = zip(*recommendations)
df = pd.DataFrame({
    'Title': titles,
    'Score': scores
})


df

,Title,Score
0,One Flew Over the Cuckoo's Nest,4.564411
1,A Close Shave,4.563151
2,The Godfather,4.561137
3,Chinatown,4.544082
4,Roger & Me,4.513459
